In [ ]:
# This Kaggle - Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


Extracting all the hashtags from twitter corpus and segmenting the words from them

In [ ]:
pip install wordsegment

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import wordsegment
from wordsegment import load, segment,clean

In [ ]:
import re
def extract_hash_tags(tweet):
    "extract hashtags from the tweet"
    return set(part[1:] for part in tweet.split() if part.startswith('#'))

import string
import pandas as pd

hashtags =[]

punct = string.punctuation
exclude_punct = set(punct)
exclude_punct_no_hash = set(punct.replace('#',''))

## read data from tweet corpus
tweets=pd.read_csv('dev_data_tweet.csv')

for twt in tweets['Tweet']: 
    "clean a tweet and extract hashtags in it"
    non_ASCII_free = (twt.encode('ascii', 'ignore')).decode("utf-8")
    punc_free = ''.join([ch for ch in non_ASCII_free if ch not in exclude_punct_no_hash])
    short_word_free = ' '.join(word for word in punc_free.split() )
    tags = extract_hash_tags(short_word_free)
    for tag in tags:
        if(len(tag)>0):
            hashtags.append(tag.lower())

##unique hashtags        
hashtags = list(set(hashtags))

print("Total unique hashtags found: {0}".format(len(hashtags)))


hashtag_columns = ['hashtag', 'word_segment']
data_frame = pd.DataFrame(columns=hashtag_columns)

mobile_brands=pd.read_csv('Mobile Brands - Sheet1.csv')
mobile_tech=[i.replace(' ','').lower() for i in mobile_brands['Brand']]


brand_models=pd.read_csv('model_brands3.csv')
mobile_tech1=[i.replace(' ','').lower() for i in brand_models['model']]
brand_models['model']=pd.Series(np.array(mobile_tech1))


load()   
    
for hashtag in hashtags:
    clean1 = clean(hashtag) 
    isegment = segment(clean1) 
    word_segment= list(isegment) 
 


    c=0
    d={}
    for i in brand_models.to_dict(orient='records'):
        if re.match(i['model'],hashtag.strip().lower()):
            c=c+1
            d[i['model']]=len(i['model'])
        else:
            continue
    if c>=1:
        key=max(d,key=d.get)
        for i in brand_models.to_dict(orient='records'):
            if i['model']==key:
                word_segment.append(i['brands'])
            else:
                continue
    else:
        for i in mobile_tech:
            if re.search(i,hashtag.strip().lower()):
                word_segment.append(i)
            else:
                continue
        
    data_frame = data_frame.append({'hashtag': "#" + hashtag, 'word_segment': set(word_segment)}, ignore_index=True)
    
segmented_Sent=[]
for i in data_frame['word_segment']:
    segmented_Sent.append(' '.join(i))
data_frame['Segmented_Sent']=pd.Series(np.array(segmented_Sent))           
            


Total unique hashtags found: 1048


In [ ]:
data_frame

,hashtag,word_segment,Segmented_Sent
0,#samsungs21ultra,"{samsung, ultra, s21}",samsung ultra s21
1,#wearables,{wearables},wearables
2,#samsunga82,"{samsung, a82}",samsung a82
3,#intensivechemotherapy,"{intensive, chemotherapy}",intensive chemotherapy
4,#experiencethefuture,"{the, experience, future}",the experience future
...,...,...,...
1043,#pnbpathshala,"{path, pnb, shala}",path pnb shala
1044,#python,{python},python
1045,#cm,{cm},cm
1046,#tigrayna,"{tigray, na}",tigray na


In [ ]:
data_frame.to_csv('HashtagSegmentation.csv')